## Notebook to test the vae model on smaller dataset

### load and sample the data

In [1]:

import sys

sys.path.append("../../")

import utils.load_csv as load_csv

set = load_csv.load_pandas()
set = load_csv.sample_data(set, 100)

# train and val only contain inliers and test contains both inliers and outliers
train, val, test = load_csv.split_data(set)
print(f"train length: {len(train)}")
print(f"val length: {len(val)}")
print(f"test length: {len(test)}")

print(f'outlier test training: {any(train["classification"] != 1)}')
print(f'outlier test validation: {any(val["classification"] != 1)}')
print(f'outlier test test: {any(test["classification"] != 1)}')
train

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


train length: 45
val length: 15
test length: 18
outlier test training: False
outlier test validation: False
outlier test test: True


,classification,file_names,label
2894,1,/home/bendm/machine_learning/project/majority-...,Singlet
2876,1,/home/bendm/machine_learning/project/majority-...,Singlet
4947,1,/home/bendm/machine_learning/project/majority-...,Singlet
5861,1,/home/bendm/machine_learning/project/majority-...,Singlet
2678,1,/home/bendm/machine_learning/project/majority-...,Singlet
7231,1,/home/bendm/machine_learning/project/majority-...,Singlet
9930,1,/home/bendm/machine_learning/project/majority-...,Singlet
971,1,/home/bendm/machine_learning/project/majority-...,Singlet
6340,1,/home/bendm/machine_learning/project/majority-...,Singlet
4640,1,/home/bendm/machine_learning/project/majority-...,Singlet


In [2]:
from models.VAE.VAE import VAE
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter
from utils.dataloader import ImagePathDataset
from torchvision.transforms import v2

2024-11-21 12:08:58.172556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732187338.183143   16703 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732187338.185824   16703 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 12:08:58.197894: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
batch_size = 8
lr = 1e-3
weight_decay = 1e-2
epochs = 80
input_dim = 4800
hidden_dim = 2000
latent_dim = 30

transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Lambda(
            lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        ),
    ]
)

train_set = ImagePathDataset(train, transform=transform)
val_set = ImagePathDataset(val, transform=transform)

train_dataloader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=True
)

In [4]:
import matplotlib.pyplot as plt
import numpy as np

for batch_idx, (data, target) in enumerate(train_dataloader):
    # check if all values are 0
    if ((data == 0).all()):
        print("nan")
        print(data)
        print(data.shape)
        plt.imshow(data[0].reshape(60, 80))
        plt.show()
        break

reasonable output after all preprocessing transformations.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=latent_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
writer = SummaryWriter(f'runs/outlier/vae_{datetime.now().strftime("%Y%m%d-%H%M%S")}')

In [ ]:
prev_updates = 0
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    prev_updates = model.train_model(train_dataloader, optimizer, prev_updates, writer)
    model.test(val_dataloader, prev_updates, writer)

Epoch 1/80


  0%|          | 0/6 [00:00<?, ?it/s]

Step 0 (N samples: 0), Loss: 3330.3960 (Recon: 3327.1709, KL: 3.2251) Grad: 27.6826


Testing: 100%|██████████| 2/2 [00:00<00:00, 33.68it/s]


====> Test set loss: 3751.5190 (BCE: 3517.5891, KLD: 233.9299)
Epoch 2/80


...

Testing: 100%|██████████| 2/2 [00:00<00:00, 52.70it/s]


====> Test set loss: 1996.1128 (BCE: 1992.6905, KLD: 3.4223)
Epoch 31/80


Testing: 100%|██████████| 2/2 [00:00<00:00, 50.20it/s]


====> Test set loss: 1993.7139 (BCE: 1990.5754, KLD: 3.1384)
Epoch 32/80


 83%|████████▎ | 5/6 [00:00<00:00,  7.50it/s]